In [2]:
!pip install nlpaug
!pip install konlpy
!pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 41.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
from konlpy.tag import Okt

# 데이터 준비
data = {
    'Q': ["안녕하세요?", "오늘 날씨는 어떤가요?", "당신의 이름은 무엇인가요?"],
    'A': ["안녕하세요. 만나서 반갑습니다.", "오늘 날씨는 맑습니다.", "저는 인공지능 챗봇입니다."]
}

df = pd.DataFrame(data)
print("Original Data:")
print(df)

# KoNLPy의 Okt 형태소 분석기 사용
okt = Okt()

# 한국어 문장을 토크나이즈하는 함수
def tokenize(text):
    return ' '.join(okt.morphs(text))

# 한국어 문장 디토크나이즈하는 함수
def detokenize(tokens):
    return ''.join(tokens)

# NLPAug 설정 (랜덤 단어 삭제)
aug = naw.RandomWordAug(action="delete")

# 증강 함수
def augment_text_korean(df, col, augmenter, n=1):
    augmented_texts = []
    for text in df[col]:
        tokens = tokenize(text)
        augmented = augmenter.augment(tokens, n=n)
        augmented_texts.append(detokenize(augmented))
    return augmented_texts

# 질문과 답변 증강
df['Q_aug'] = augment_text_korean(df, 'Q', aug)
df['A_aug'] = augment_text_korean(df, 'A', aug)

print("\nAugmented Data:")
print(df[['Q_aug', 'A_aug']])


Original Data:
                Q                  A
0          안녕하세요?  안녕하세요. 만나서 반갑습니다.
1    오늘 날씨는 어떤가요?       오늘 날씨는 맑습니다.
2  당신의 이름은 무엇인가요?     저는 인공지능 챗봇입니다.

Augmented Data:
       Q_aug      A_aug
0          ?   . 반갑습니다.
1  오늘 날씨 어떤?      날씨 는.
2   당신 의 이름?  는 인공 입니다.


In [5]:
import pandas as pd
import nlpaug.augmenter.word as naw
from konlpy.tag import Okt

# 데이터 로드
file_path = '/content/drive/MyDrive/Colab Notebooks/makedata.csv'
df = pd.read_csv(file_path)

# 데이터 확인
print("Original Data:")
print(df.head())

# KoNLPy의 Okt 형태소 분석기 사용
okt = Okt()

# 한국어 문장을 토크나이즈하는 함수
def tokenize(text):
    return okt.morphs(text)

# 한국어 문장 디토크나이즈하는 함수
def detokenize(tokens):
    return ' '.join(tokens)

# NLPAug 설정 (랜덤 단어 삭제)
aug = naw.RandomWordAug(action="delete")

# 증강 함수
def augment_text_korean(text, augmenter, n=1):
    tokens = tokenize(text)
    augmented = augmenter.augment(tokens, n=n)
    return detokenize(augmented)

# 데이터 증강
df['Q_aug'] = df['Q'].apply(lambda x: augment_text_korean(x, aug))
df['A_aug'] = df['A'].apply(lambda x: augment_text_korean(x, aug))

# 증강된 데이터 확인
print("\nAugmented Data:")
print(df[['Q', 'Q_aug', 'A', 'A_aug']].head())


Original Data:
                                 Q                          A  id
0               오늘 저녁에 오랜만에 영화 고고?                오우 너무 좋지? ㅎ   1
1                        몇 시에 퇴근해?        난 3시면 퇴근하지요 ㅎ 자기는 ?   2
2  나는 5시 인데 회사 일이 많아서 조금 더 늦을 수 있어     그럼 영화관 가기는 그렇고 집에서 볼까?   3
3           좋지 좋지 ㅎ 집에서 치맥 하면서 볼까?  너무 좋지 하하 상상만 해도 벌써 힐링인데 ㅎ   4
4           근데 자기 다이어트 한다며 ㅋ 괜찮겠어?             맛있으면 0칼로리라잖아 ㅎ   5

Augmented Data:
                                 Q                              Q_aug  \
0               오늘 저녁에 오랜만에 영화 고고?             오늘 저녁 에 오랜 만 에 영화 고고 ?   
1                        몇 시에 퇴근해?                       몇 시 에 퇴근 해 ?   
2  나는 5시 인데 회사 일이 많아서 조금 더 늦을 수 있어  나 는 5시 인데 회사 일이 많아서 조금 더 늦 을 수 있어   
3           좋지 좋지 ㅎ 집에서 치맥 하면서 볼까?           좋지 좋지 ㅎ 집 에서 치맥 하면서 볼까 ?   
4           근데 자기 다이어트 한다며 ㅋ 괜찮겠어?            근데 자기 다이어트 한다며 ㅋ 괜찮겠어 ?   

                           A                        A_aug  
0                오우 너무 좋지? ㅎ                 오우 너무 좋지 ? ㅎ  
1        난 3시

In [6]:
import pandas as pd
import nlpaug.augmenter.word as naw
from konlpy.tag import Okt

# 데이터 로드
file_path = '/content/drive/MyDrive/Colab Notebooks/makedata.csv'
df = pd.read_csv(file_path)

# 데이터 확인
print("Original Data:")
print(df.head())

# KoNLPy의 Okt 형태소 분석기 사용
okt = Okt()

# 한국어 문장을 토크나이즈하는 함수
def tokenize(text):
    return okt.morphs(text)

# 한국어 문장 디토크나이즈하는 함수
def detokenize(tokens):
    return ' '.join(tokens)

# NLPAug 설정 (랜덤 단어 삭제)
aug = naw.RandomWordAug(action="delete")

# 증강 함수
def augment_text_korean(text, augmenter, n=1):
    tokens = tokenize(text)
    augmented = augmenter.augment(tokens, n=n)
    if isinstance(augmented, list):
        augmented = augmented[0]
    return detokenize(augmented)

# 데이터 증강
df['Q_aug'] = df['Q'].apply(lambda x: augment_text_korean(x, aug))
df['A_aug'] = df['A'].apply(lambda x: augment_text_korean(x, aug))

# 증강된 데이터 확인
print("\nAugmented Data:")
print(df[['Q', 'Q_aug', 'A', 'A_aug']].head())

Original Data:
                                 Q                          A  id
0               오늘 저녁에 오랜만에 영화 고고?                오우 너무 좋지? ㅎ   1
1                        몇 시에 퇴근해?        난 3시면 퇴근하지요 ㅎ 자기는 ?   2
2  나는 5시 인데 회사 일이 많아서 조금 더 늦을 수 있어     그럼 영화관 가기는 그렇고 집에서 볼까?   3
3           좋지 좋지 ㅎ 집에서 치맥 하면서 볼까?  너무 좋지 하하 상상만 해도 벌써 힐링인데 ㅎ   4
4           근데 자기 다이어트 한다며 ㅋ 괜찮겠어?             맛있으면 0칼로리라잖아 ㅎ   5

Augmented Data:
                                 Q Q_aug                          A    A_aug
0               오늘 저녁에 오랜만에 영화 고고?   오 늘                오우 너무 좋지? ㅎ      오 우
1                        몇 시에 퇴근해?     몇        난 3시면 퇴근하지요 ㅎ 자기는 ?        난
2  나는 5시 인데 회사 일이 많아서 조금 더 늦을 수 있어     나     그럼 영화관 가기는 그렇고 집에서 볼까?      그 럼
3           좋지 좋지 ㅎ 집에서 치맥 하면서 볼까?   좋 지  너무 좋지 하하 상상만 해도 벌써 힐링인데 ㅎ      너 무
4           근데 자기 다이어트 한다며 ㅋ 괜찮겠어?   근 데             맛있으면 0칼로리라잖아 ㅎ  맛 있 으 면


In [ ]:
if isinstance(augmented, list):
    augmented = augmented[0]
